In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from operations import adjust_dataset, split_data, Ann


In [2]:
df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')
target_variable = 'temperature_celsius'
# target_variable = 'cloud'
df = adjust_dataset(df, target_variable=target_variable)

print (df.shape)

# for temperature in df['temperature_celsius'].unique():
#     print (temperature)




(18904, 48)


In [3]:
model = Ann(df, target_variable)
model.train_model(epochs=50, batch_size=32)

In [4]:
res = model.evaluate()
print (res)

3.117969036102295


# Feature importance

In [ ]:

# Compute feature importance
feature_importance_values = model.compute_feature_importance()

sorted_feature_importance = sorted(feature_importance_values, key=lambda x: x[1], reverse=True)

# Iterate over the sorted list and print column name and value
for column, value in sorted_feature_importance:
    print(f"{column} --> {value}")


wind_direction_SE --> 2.7557991870708065e-06
moon_illumination --> 2.4974428924906533e-06
wind_direction_WNW --> 2.4974428924906533e-06
moon_phase_Waning Gibbous --> 2.4974428924906533e-06
uv_index --> 2.3252055143530015e-06
moon_phase_Full Moon --> 2.2390868252841756e-06
air_quality_Ozone --> 2.1529681362153497e-06
moon_phase_Last Quarter --> 2.1529681362153497e-06
air_quality_gb-defra-index --> 2.0668492197728483e-06
wind_direction_WSW --> 2.0668492197728483e-06
moon_phase_Waxing Crescent --> 2.0668492197728483e-06
wind_direction_ESE --> 1.9807305307040224e-06
wind_direction_NNE --> 1.8946118416351965e-06
moon_phase_Waxing Gibbous --> 1.636255660741881e-06
wind_direction_W --> 1.550136971673055e-06
precip_mm --> 1.4640182826042292e-06
moon_phase_First Quarter --> 1.4640182826042292e-06
moon_phase_Waning Crescent --> 1.4640182826042292e-06
feels_like_celsius --> 1.3778995935354033e-06
air_quality_Carbon_Monoxide --> 1.3778995935354033e-06
wind_direction_SSW --> 1.3778995935354033e-06


# Hyperparameter tuning


In [ ]:
hyperparameter_grid = {
    'lr': [0.001],
    'batch_size': [32],
    'dropout': [0.2],
    'max_gradient_norm': [1,2,3,4],
    'lr_schedule': ['constant', 'step_decay', 'exponential_decay'],
    'optimizer': ['SGD', 'Adam', 'RMSprop']
}
num_epochs_list = [50]

tuning_results = model.hyperparameter_tuning(hyperparameter_grid, num_epochs_list=num_epochs_list)

print (tuning_results)

{'params': {'batch_size': 32, 'dropout': 0.2, 'lr': 0.001, 'lr_schedule': 'step_decay', 'max_gradient_norm': 2, 'optimizer': 'Adam'}, 'num_epochs': 50, 'evaluation_result': 4.224359035491943}
